In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

In [2]:
# 파이토치 프레임워크 불러오는 부분
import torch
import torchvision # image processing에 특화된 라이브러리
import torch.nn as nn # neural network
import torchvision.transforms as transforms # Data Augmentation 특화된 라이브러리
import numpy as np

In [6]:
X = torch.Tensor(2,3) # 2행 3열의 텐서를 하나 생성
X.shape
X.size()

X = torch.Tensor([[1,2,3],[4,5,6]])
X

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [ ]:
X = torch.Tensor(2,3)
X.shape
X.size()

X = torch.Tensor([[1,2,3],[4,5,6]])
X

In [8]:
'''
tensor()인자값으로
data : 값지정, 초기화
dtype
requires_grad : tensor에 대해서 미분 적용할지 여부
'''

x = torch.tensor(data=[2.0,3.0], requires_grad=True) # 입력값
x

y = x**2 # 4, 9 
y

pred = 2*y + 3 # 11, 21 -> 예측값

target = torch.tensor([3.0,4.0]) # 3, 4 -> 타겟
target

#예측에서 정답을 빼면 loss
#loss = (11-3)+(21-4) = 25

tensor([3., 4.])

In [ ]:
x = torch.tensor(data=[2.0,3.0], requires_grad=True)
x

y = x**2
y

pred = 2*y + 3

target = torch.tensor([3.0,4.0])
target

In [ ]:
'''
위에 나온 결과를 바탕으로
경사하강법을 이용해서 
미분을 사용해서 Loss에 대한 책임을 묻겠다
기울기가 계산이 될 것이다.(미분 계산)
--> BackPropagation 을 적용하려면 backward() 함수를 호출

Loss를 줄여나가 보겠다.
'''

In [9]:
loss = torch.sum(torch.abs(pred - target))
print('Loss',loss)

Loss tensor(25., grad_fn=<SumBackward0>)


In [10]:
loss.backward()

In [11]:
print(x.grad,pred.grad) # 기울기 계산이 안됨

tensor([ 8., 12.]) None


<ipython-input-11-f0cd9ca8c1fa>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print(x.grad,pred.grad)


### Create Tesor...Using NeuralNet

In [12]:
x = torch.randn(10,3)
y = torch.randn(10,2)

print(x)
print(y)

tensor([[-0.2874, -1.5762, -1.5403],
        [ 0.5376,  2.0259, -0.8607],
        [ 1.0645, -0.1771, -0.9113],
        [ 1.2170, -0.3671, -1.0788],
        [ 1.0456,  0.9344,  0.0431],
        [ 0.2603, -0.4910,  0.3295],
        [-0.6879,  1.3867, -0.2419],
        [ 0.3960,  0.8492,  0.2218],
        [ 0.8542, -0.8396, -0.9596],
        [-0.2468,  0.5085, -0.3830]])
tensor([[-0.4198, -0.0186],
        [-0.6257, -0.8362],
        [-0.3832, -0.2261],
        [-0.6525,  1.6509],
        [ 0.4967, -1.0812],
        [-0.1631, -0.0754],
        [ 1.7628, -0.3099],
        [ 1.0655, -0.4675],
        [ 0.0854, -2.1135],
        [ 0.8530,  0.2630]])


In [13]:
linear = nn.Linear(3,2)
print("w",linear.weight)
print("b",linear.bias)

w Parameter containing:
tensor([[ 0.3471, -0.5402,  0.0516],
        [ 0.5423, -0.1827,  0.2582]], requires_grad=True)
b Parameter containing:
tensor([0.1059, 0.4946], requires_grad=True)


In [19]:
# 객체 출력, parameters()함수는 모델 안의 학습의 주체인 w,b를 내포하고 있는 객체
# parameters() 함수는 w,b를 해킹한 함수
print(linear.parameters())

# loss function을 미리 선정의 해두었다.
loss_function = nn.MSELoss()

# optimizer(하산하는 방법)를 미리 정의
optimizer = torch.optim.SGD(linear.parameters(),lr=0.01)

# 위에서 만든 모델에 값을 입력...결과로 예측값이 나온다.
pred = linear(x)

# 정답과 예측값을 이용해서 위에서 선정의한 Loss를 계산...L(W)
loss = loss_function(pred, y)
print("loss before step BackPropagation",loss.item())

<generator object Module.parameters at 0x000001E73EB4E4A0>
loss before step BackPropagation 1.6401102542877197


In [20]:
'''
loss값이 나왔다는 것은
모델의 예측값에 대한 잘잘못을 정량화 했다는 것이다.

이 값을 바탕으로 BackPropagation을 하면 w, b에 대한 미분값
즉 얼마만큼 보정해야 하는지의 값이 나온다.
그 값을 이용해서 다시 하강(기울기 수정)하기 때문에
2번쨰에는 Loss가 줄어들어야 한다.
'''

loss.backward() # loss값에 대한 w에 책임을 묻는다...미분을 적용. grad
print("dL/dw", linear.weight.grad)
print("dL/db", linear.bias.grad)

dL/dw tensor([[ 0.3946, -0.9733, -0.3620],
        [ 0.6005,  0.1710, -0.3734]])
dL/db tensor([-0.1013,  0.8611])


In [21]:
optimizer.step() # 위에서 수정된 값으로 하강을 진행한다...학습을 한다

# 반복효과
pred = linear(x)
loss = loss_function(pred,y)

print("loss after BackPropagation : ",loss.item())

loss after BackPropagation :  1.6151301860809326
